In [1]:
library(bnlearn)


Attaching package: 'bnlearn'

The following object is masked from 'package:stats':

    sigma



In [2]:
### Si données locales
u.user <- read.csv(file='u.user.csv', sep='|', header=T)
u.item <- read.csv(file='u.item.csv', sep='|', header=T)
u.data <- read.csv(file='u.data.csv', sep='|', header=T)
library(Matrix)
m.sparse <- sparseMatrix(u.data[,1],u.data[,2],x=u.data[,3])
rownames(m.sparse) <- paste('u', 1:nrow(m.sparse), sep='')
colnames(m.sparse) <- paste('i', 1:ncol(m.sparse), sep='')
mae <- function(m1, m2) mean(abs(m1 - m2), na.rm=T)

m <- as.matrix(m.sparse)                # transforme en matrice ordinaire
m.na <- m
m.na[m.na==0] <- NA
mae.rmse <- function(m, m.hat) {
    return(list(mae=mean(abs(m - m.hat), na.rm=T), rmse=sqrt(mean((m - m.hat)^2, na.rm=T))))
}
new.m.mae.rmse <- function(dimRedu){
# matrice diagonale pour réduire les matrices originales
m.d2 <- diag(c(m.svd$d[1:dimRedu],rep(0,length(m.svd$d)-dimRedu)))
# nouvelle matrice basée sur les deux premières dimensions uniquement
m.svd2 <- round((m.svd$u %*% m.d2 %*% t(m.svd$v)),2)
mae.rmse(m.svd2, m)
    }

In [3]:
# merge users and ratings
names(u.user)<- c('user.id','age','gender','job')
u.user <- u.user[,c(-5)]
cat.age <- function(x) floor(x/20)
u.user[, 2] <- sapply(u.user[, 2], cat.age)
for (i in 2:4){
u.user[, i] <- sapply(u.user[, i], as.factor)
}
head(u.user)
names(u.data)<- c('user.id','movie.id','rating')
u.data <- u.data[,c(-4)]
u.data[, 3] <- sapply(u.data[, 3], as.factor)
head(u.data)
u.item <- u.item[,c(-2,-3,-4,-5)]
for (i in 2:20){
u.item[, i] <- sapply(u.item[, i], as.factor)
}
head(u.item)
u.r <- merge(u.user, u.data, by = "user.id")
# merge userRating with movie to have actual movie titles instead of ids
u.r.v <- merge(u.r, u.item, by = "movie.id")
head(u.r.v)

user.id,age,gender,job
1,1,M,technician
2,2,F,other
3,1,M,writer
4,1,M,technician
5,1,F,other
6,2,M,executive


user.id,movie.id,rating
196,242,3
186,302,3
22,377,1
244,51,2
166,346,1
298,474,4


movie.id,unknown,Action,Adventure,Animation,Children.s,Comedy,Crime,Documentary,Drama,Fantasy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


movie.id,user.id,age,gender,job,rating,unknown,Action,Adventure,Animation,...,Fantasy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
1,918,2,M,scientist,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,684,1,M,student,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,660,1,M,student,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,339,1,M,lawyer,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,699,2,M,other,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,94,1,M,student,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
new.m.cv <- function(dimRedu){
# matrice diagonale pour réduire les matrices originales
m.d2 <- diag(c(m.svd$d[1:dimRedu],rep(0,length(m.svd$d)-dimRedu)))
# nouvelle matrice basée sur les deux premières dimensions uniquement
m.svd2 <- round((m.svd$u %*% m.d2 %*% t(m.svd$v)),2)
mae(m.svd2[i.test.c]*5, m.na[i.test.c])
    }
to.plot <- c()

In [ ]:
set.seed(5)
i.observed <- dim(u.r.v)[1]
i.hasard <- sample(seq(1,i.observed,1), i.observed)
fold.size <- round(length(i.hasard) / 5)
i.false <- rep(FALSE, dim(u.r.v)[1])
for (i in seq_len(5)){
fold.number <- i

## Index booléen pour les cellules de test et d'entraînement
i.test.b <- i.false
## Les cellules indexées du replis correspondant sont fixées à TRUE pour le test...
i.test.b[ i.hasard[((fold.number-1) * fold.size):((fold.number) * fold.size)] ] <- TRUE
a<-as.data.frame(u.data[1,3])
colnames(a)<-'rating'   

## ...et à FALSE pour l'entraînement
i.train.b <-  !i.test.b
i.test.c <- rep(FALSE,943*1682)
u.r.v.train <- u.r.v[i.train.b,]
u.r.v.test <- u.r.v[i.test.b,]
nb <- naive.bayes(u.r.v.train[,3:25],'rating')
fitted <- bn.fit(nb,u.r.v.train[,3:25] )
new.m <- m
i.test.c <- rep(FALSE,943*1682)
for (j in seq_len(fold.size)){
    new.m[u.r.v.test[j,2],u.r.v.test[j,1]]<-0
    i.test.c[943*(u.r.v.test[j,1]-1)+u.r.v.test[j,2]]<-TRUE
}
for (k in seq_len(943)){
    for (l in seq_len(1682)){
        if (new.m[k,l]==0){
            allo <-data.frame(cbind(u.item[l,-1],a,u.user[k,-1]))
            new.m[k,l]<-as.numeric(predict(fitted, allo))
        }
    }
}
write.table(new.m,file=paste("naiveBayes1_", toString(i), ".csv", sep = "")) # drops the rownames
write.table(i.test.c,file=paste("naiveBayesTestc1_", toString(i), ".csv", sep = "")) # drops the rownames
write.table(i.test.b,file=paste("naiveBayesTestb1_", toString(i), ".csv", sep = "")) # drops the rownames

m.train <- new.m
m.train <- m.train/5
m.svd <- svd(m.train)
to.plot<-rbind(to.plot,unlist(lapply(seq(75), new.m.cv)))
    }


In [ ]:
to.plot<-apply(to.plot,2,unlist)

In [ ]:
sdev <- apply(to.plot,2,sd) 
sdev
avg <- colMeans(to.plot)
x <- 1:length(avg)
avg

In [ ]:
plot(x,colMeans(to.plot),ylim=range(c(avg-sdev, avg+sdev)),pch=19, xlab="Measurements", ylab="Mean +/- SD",
    main="Scatter plot with std.dev error bars"
)
# hack: we draw arrows but with very special "arrowheads"
arrows(x, avg-sdev, x, avg+sdev, length=0.05, angle=90, code=3)

In [ ]:
i.observed <- dim(u.r.v)[1]
i.hasard <- sample(seq(1,i.observed,1), i.observed)
fold.size <- round(length(i.hasard) / 5)
i.false <- rep(FALSE, dim(u.r.v)[1])
fold.number <- 1
a<-as.data.frame(u.data[1,3])
colnames(a)<-'rating'
## Index booléen pour les cellules de test et d'entraînement
i.test.b <- i.false
## Les cellules indexées du replis correspondant sont fixées à TRUE pour le test...
i.test.b[ i.hasard[((fold.number-1) * fold.size):((fold.number) * fold.size)] ] <- TRUE
    

## ...et à FALSE pour l'entraînement
i.train.b <-  !i.test.b
u.r.v.train <- u.r.v[i.train.b,]
u.r.v.test <- u.r.v[i.test.b,]
nb <- naive.bayes(u.r.v.train[,3:25],'rating')
fitted <- bn.fit(nb,u.r.v.train[,3:25] )
new.m <- m
for (j in seq_len(fold.size)){
    new.m[u.r.v.test[j,2],u.r.v.test[j,1]]<-0
}
for (k in seq_len(943)){
    for (l in seq_len(1682)){
        if (new.m[k,l]==0){
            allo <-data.frame(cbind(u.item[l,-1],a,u.user[k,-1]))
            new.m[k,l]<-as.numeric(predict(fitted, allo))
        }
    }
}
write.table(new.m,file=paste("naiveBayes1_", toString(i), ".csv", sep = "")) # drops the rownames
m.train <- new.m
m.train <- m.train/5
m.svd <- svd(m.train)
to.plot<-rbind(to.plot,unlist(lapply(seq(22), new.m.cv)))
    


In [25]:
asdf<-read.table("naiveBayes1_5.csv", sep = "") # drops the rownames

In [26]:
dim(asdf)

[1]  943 1682

In [27]:
mae(asdf,m.na)

Warning message in mean.default(abs(m1 - m2), na.rm = T):
"argument is not numeric or logical: returning NA"

[1] NA

In [28]:
mean(abs(asdf-m.na),na.rm=T)

Warning message in mean.default(abs(asdf - m.na), na.rm = T):
"argument is not numeric or logical: returning NA"

[1] NA

In [31]:
mean((abs(asdf[i.test.b]-m.na[i.test.b])),na.rm=T)

ERROR: Error in `[.data.frame`(asdf, i.test.b): undefined columns selected


In [ ]:
m.train <- m.train/5
m.svd <- svd(m.train)
to.plot<-rbind(to.plot,unlist(lapply(seq(22), new.m.cv)))

In [32]:
(abs(asdf[i.test.b]-m.na[i.test.b]))

ERROR: Error in `[.data.frame`(asdf, i.test.b): undefined columns selected
